In [1]:
import os

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.utils import save_image

from tqdm import tqdm
import matplotlib.pyplot as plt
import copy
import numpy as np

In [2]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()

        self.fc1 = nn.Linear(latent_dim, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 1024)
        self.fc5 = nn.Linear(1024, 784)

        self.BN1 = nn.BatchNorm1d(256)
        self.BN2 = nn.BatchNorm1d(512)
        self.BN3 = nn.BatchNorm1d(1024)
        
        self.leaky_relu1 = nn.LeakyReLU(0.2)
        self.leaky_relu2 = nn.LeakyReLU(0.2)
        self.leaky_relu3 = nn.LeakyReLU(0.2)
        self.leaky_relu4 = nn.LeakyReLU(0.2)


        # Construct generator. You should experiment with your model,
        # but the following is a good start:
        #   Linear args.latent_dim -> 128
        #   LeakyReLU(0.2)
        
        #   Linear 128 -> 256
        #   Bnorm
        #   LeakyReLU(0.2)
        
        #   Linear 256 -> 512
        #   Bnorm
        #   LeakyReLU(0.2)
        
        #   Linear 512 -> 1024
        #   Bnorm
        #   LeakyReLU(0.2)
        
        #   Linear 1024 -> 784
        #   Output non-linearity

    def forward(self, z):
        # Generate images from z
        z = self.leaky_relu1(self.fc1(z))
        z = self.leaky_relu2(self.BN1(self.fc2(z)))
        z = self.leaky_relu3(self.BN2(self.fc3(z)))
        z = self.leaky_relu4(self.BN3(self.fc4(z)))
        z = torch.tanh(self.fc5(z))
        return z


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.leaky_relu1 = nn.LeakyReLU(0.2)
        self.leaky_relu2 = nn.LeakyReLU(0.2)

        # Construct distriminator. You should experiment with your model,
        # but the following is a good start:
        #   Linear 784 -> 512
        #   LeakyReLU(0.2)
        #   Linear 512 -> 256
        #   LeakyReLU(0.2)
        #   Linear 256 -> 1
        #   Output non-linearity

    def forward(self, img):
        # return discriminator score for img
        img = self.leaky_relu1(self.fc1(img))
        img = self.leaky_relu2(self.fc2(img))
        img = torch.sigmoid(self.fc3(img))
        return img

In [3]:
max_epoch = 200
batch_size = 64
learning_rate = 0.0002
latent_dim = 100
save_interval = 500
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/usr/local/miniconda3/lib/python3.8/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
def train(dataloader, discriminator, generator, optimizer_G, optimizer_D):
    criterion = nn.BCELoss()
    for epoch in tqdm(range(max_epoch)):
        discriminator.train()
        generator.train()
        G_total_loss = 0.
        D_total_loss = 0.
        for i, (imgs, _) in enumerate(dataloader):

            imgs.to(device)
            imgs_real = torch.flatten(imgs, start_dim=1).to(device)
            
            batch_size = imgs_real.size(0)

            

            # Train Discriminator
            # -------------------
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_images = generator(z)
            
            optimizer_D.zero_grad()

            real_preds = discriminator(imgs_real)
            real_labels = torch.ones(batch_size, 1).to(device)
            real_loss = criterion(real_preds, real_labels)

            fake_preds = discriminator(fake_images)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            fake_loss = criterion(fake_preds, fake_labels)


            d_loss = real_loss + fake_loss
            d_loss.backward()
            optimizer_D.step()


            # Train Generator
            # ---------------
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_images = generator(z)
            

            optimizer_G.zero_grad()

            fake_preds = discriminator(fake_images)
            real_labels = torch.ones(batch_size, 1).to(device)

            g_loss = criterion(fake_preds, real_labels)
            g_loss.backward()
            optimizer_G.step()

            imgs_gen = 0.5 * (fake_images + 1).view(batch_size, 1, 28, 28)

            batches_done = epoch * len(dataloader) + i
            if batches_done % save_interval == 0:
                # You can use the function save_image(Tensor (shape Bx1x28x28),
                # filename, number of rows, normalize) to save the generated
                # images, e.g.:
                save_image(imgs_gen[:25],'images/{}.png'.format(batches_done),nrow=5, normalize=True, value_range=(-1,1))
                print("image saved")
                

            if (i+1) % 200 == 0:
                print(f"Epoch [{epoch+1}/{max_epoch}] Batch {i+1}/{len(dataloader)} "
                      f"Loss D: {d_loss:.4f}, Loss G: {g_loss:.4f}")

In [5]:
os.makedirs('images', exist_ok=True)

In [6]:
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/mnist', train=True, download=True,
                   transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize(0.5, 0.5)])),
    batch_size=batch_size, shuffle=True)

In [7]:
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
optimizer_G = torch.optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

In [ ]:
train(dataloader, discriminator, generator, optimizer_G, optimizer_D)

  0%|          | 0/200 [00:00<?, ?it/s]

image saved
Epoch [1/200] Batch 200/938 Loss D: 0.4533, Loss G: 4.6778
Epoch [1/200] Batch 400/938 Loss D: 0.1854, Loss G: 5.3930
image saved
Epoch [1/200] Batch 600/938 Loss D: 0.5350, Loss G: 7.4200
Epoch [1/200] Batch 800/938 Loss D: 0.2975, Loss G: 4.1808


  0%|          | 1/200 [01:08<3:47:19, 68.54s/it]

image saved
Epoch [2/200] Batch 200/938 Loss D: 0.2831, Loss G: 5.8344
Epoch [2/200] Batch 400/938 Loss D: 0.3017, Loss G: 6.4607
image saved
Epoch [2/200] Batch 600/938 Loss D: 0.1448, Loss G: 5.8045
Epoch [2/200] Batch 800/938 Loss D: 0.3214, Loss G: 4.1796


  1%|          | 2/200 [02:10<3:34:15, 64.92s/it]

...

Epoch [184/200] Batch 200/938 Loss D: 1.2868, Loss G: 1.1980
image saved
Epoch [184/200] Batch 400/938 Loss D: 1.1176, Loss G: 1.1894
Epoch [184/200] Batch 800/938 Loss D: 1.1259, Loss G: 0.9666
image saved


 92%|█████████▏| 184/200 [3:18:10<14:37, 54.86s/it]

Epoch [185/200] Batch 200/938 Loss D: 1.3560, Loss G: 1.3130
Epoch [185/200] Batch 400/938 Loss D: 1.1662, Loss G: 1.1161
image saved
Epoch [185/200] Batch 600/938 Loss D: 1.1516, Loss G: 0.9820
Epoch [185/200] Batch 800/938 Loss D: 1.1925, Loss G: 1.0619
image saved


 92%|█████████▎| 185/200 [3:19:01<13:27, 53.82s/it]

Epoch [186/200] Batch 200/938 Loss D: 1.1757, Loss G: 1.1722
Epoch [186/200] Batch 400/938 Loss D: 1.1441, Loss G: 1.0279
image saved
Epoch [186/200] Batch 600/938 Loss D: 1.3015, Loss G: 1.0476
Epoch [186/200] Batch 800/938 Loss D: 1.1231, Loss G: 0.9920
